In [1]:
import os
import random
import tensorflow as tf

from pathlib import Path
from tensorflow.keras.applications import resnet50
from tensorflow.keras import (Sequential, preprocessing, layers, optimizers, Model)

In [2]:
DATA_DIRECTORY = Path('/content/drive/MyDrive/data/flowers_dataset')
TRAIN_DATA = DATA_DIRECTORY / "train"
TEST_DATA = DATA_DIRECTORY / "test"

IMAGE_SIZE = 128
BATCH_SIZE = 32
TRAIN_RATIO = 0.8
EPOCHS = 10

In [3]:
def parse_image(filename):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    return image

In [4]:
def _datasets(data_directory):
    classes = os.listdir(data_directory)
    
    if ".DS_Store" in classes:
        classes.remove(".DS_Store")

    filenames = list(data_directory.glob("**/*.jpg"))
    random.shuffle(filenames)

    labels = [classes.index(str(name).split("/")[-2]) for name in filenames]
    num_classes = max(labels) + 1
    labels_onehot = tf.one_hot(labels, num_classes)

    filenames = [str(name) for name in filenames]

    files_dataset = tf.data.Dataset.from_tensor_slices(filenames)
    image_count = files_dataset.cardinality().numpy()

    images_dataset = files_dataset.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    labels_dataset = tf.data.Dataset.from_tensor_slices(labels_onehot)

    dataset = tf.data.Dataset.zip((images_dataset, labels_dataset))
    dataset = dataset.shuffle(buffer_size=50)

    train_dataset = dataset.take(image_count * TRAIN_RATIO)
    validation_dataset = dataset.skip(image_count * TRAIN_RATIO)

    train_dataset = train_dataset.batch(BATCH_SIZE)
    train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    validation_dataset = validation_dataset.batch(BATCH_SIZE)
    validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return train_dataset, validation_dataset, num_classes

In [5]:
def _model(num_classes):
    data_augmentation = Sequential([
        layers.RandomFlip("horizontal", input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), seed=32),
        layers.RandomRotation(factor=(-0.1, 0.1), seed=42),
        layers.RandomZoom(height_factor=(-0.1, 0.1), width_factor=(-0.1, 0.1), seed=42),
        layers.RandomTranslation(height_factor=(-0.05, 0.05), width_factor=(-0.05, 0.05), seed=32)
    ])

    base_model = resnet50.ResNet50(
        weights="imagenet",
        include_top=False
    )

    image_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="image")
    x = data_augmentation(image_input)
    x = resnet50.preprocess_input(x)
    x = base_model(x, training=False)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = Model(inputs=image_input, outputs=outputs)

    print(model.summary())

    return model

In [6]:
train_set, val_set, num_classes = _datasets(TRAIN_DATA)

In [7]:
model = _model(num_classes)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 128, 128, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 128, 128, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 128, 128, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 128, 128, 3)      0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 dropout (Dropout)           (None, 4, 4, 2048)        0         
                                                             

In [8]:
model.compile(optimizer=optimizers.Adam(), 
            loss="categorical_crossentropy",
            metrics=["accuracy"]
)

In [9]:
model.fit(train_set, validation_data=val_set, epochs = EPOCHS, verbose=1)

Epoch 1/10
68/68 [==============================] - 40s 434ms/step - loss: 1.9530 - accuracy: 0.2325 - val_loss: 1.6105 - val_accuracy: 0.1908
Epoch 2/10
68/68 [==============================] - 28s 410ms/step - loss: 1.6128 - accuracy: 0.2381 - val_loss: 1.6080 - val_accuracy: 0.2055
Epoch 3/10
68/68 [==============================] - 28s 409ms/step - loss: 1.6055 - accuracy: 0.2353 - val_loss: 1.6015 - val_accuracy: 0.2055
Epoch 4/10
68/68 [==============================] - 28s 408ms/step - loss: 1.5684 - accuracy: 0.2707 - val_loss: 1.5393 - val_accuracy: 0.2771
Epoch 5/10
68/68 [==============================] - 28s 408ms/step - loss: 1.8542 - accuracy: 0.2298 - val_loss: 1.6090 - val_accuracy: 0.1982
Epoch 6/10
68/68 [==============================] - 28s 409ms/step - loss: 1.6066 - accuracy: 0.2390 - val_loss: 1.6081 - val_accuracy: 0.2092
Epoch 7/10
68/68 [==============================] - 28s 408ms/step - loss: 1.6036 - accuracy: 0.2376 - val_loss: 1.6098 - val_accuracy: 0.1945